<a href="https://colab.research.google.com/github/ttury/Deep-Learning-For-Natural-Language-Processing/blob/master/DeeDee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# transformer.py 파일 업로드 필요

from google.colab import files

src = list(files.upload().values())[0]
open('module.py','wb').write(src)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
import time
import tensorflow_datasets as tfds
import tensorflow as tf
from transformer import *

urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData%20.csv", filename="ChatBotData.csv")
train_data = pd.read_csv('ChatBotData.csv')

print(len(train_data))
print(train_data.isnull().sum())
train_data.head()

11823
Q        0
A        0
label    0
dtype: int64


,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [3]:
questions = list()
for sentence in train_data['Q']:
  sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
  sentence = sentence.strip() # 양쪽 공백 제거
  questions.append(sentence)

answers = list()
for sentence in train_data['A']:
  sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
  sentence = sentence.strip() # 양쪽 공백 제거
  answers.append(sentence)

print(questions[:5])
print(answers[:5])

['12시 땡 !', '1지망 학교 떨어졌어', '3박4일 놀러가고 싶다', '3박4일 정도 놀러가고 싶다', 'PPL 심하네']
['하루가 또 가네요 .', '위로해 드립니다 .', '여행은 언제나 좋죠 .', '여행은 언제나 좋죠 .', '눈살이 찌푸려지죠 .']


In [4]:
# 서브워드텍스트인코더를 이용해 단어 사전 생성

tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    questions + answers, target_vocab_size=2**13
)

# 단어 사전에 인덱스 2개 추가(디코더의 입력에 사용하는 SOS, EOS 토큰)
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]
VOCAB_SIZE = tokenizer.vocab_size + 2

print('SOS 토큰 번호 :', START_TOKEN)
print('EOS 토큰 번호 :', END_TOKEN)
print('단어 집합의 크기 :', VOCAB_SIZE)
# 패딩 토큰 + 8179번 토큰 = 단어 집합의 크기

SOS 토큰 번호 : [8178]
EOS 토큰 번호 : [8179]
단어 집합의 크기 : 8180


In [5]:
# 서브워드텍스트인코더 토크나이저의 encode()를 사용하여 정수 인코딩

sample_string = questions[20]

tokenized_string = tokenizer.encode(sample_string)
print('정수 인코딩 후의 문장 : {}'.format(tokenized_string))

original_string = tokenizer.decode(tokenized_string)
print('기존 문장 : {}'.format(original_string))

for token in tokenized_string:
  print ('{} ----> {}'.format(token, tokenizer.decode([token])))
  # decode의 경우 리스트를 인자로 받는다
  # 한 어절이 정수 인코딩 후에는 여러 서브워드로 맵핑될 수 있음

정수 인코딩 후의 문장 : [5766, 611, 3509, 141, 685, 3747, 849]
기존 문장 : 가스비 비싼데 감기 걸리겠어
5766 ----> 가스
611 ----> 비 
3509 ----> 비싼
141 ----> 데 
685 ----> 감기 
3747 ----> 걸리
849 ----> 겠어


In [6]:
MAX_LENGTH = 40
set_max_length(MAX_LENGTH)

def tokenize_and_filter(inputs, outputs):
  tokenized_inputs, tokenized_outputs = [], []

  for (sent1, sent2) in zip(inputs, outputs):
    sent1 = START_TOKEN + tokenizer.encode(sent1) + END_TOKEN
    sent2 = START_TOKEN + tokenizer.encode(sent2) + END_TOKEN

    tokenized_inputs.append(sent1)
    tokenized_outputs.append(sent2)
  
  tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_inputs, maxlen=MAX_LENGTH, padding='post') # 뒤를 패딩
  tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_outputs, maxlen=MAX_LENGTH, padding='post')
  
  return tokenized_inputs, tokenized_outputs

questions, answers = tokenize_and_filter(questions, answers)

In [7]:
print('Q 데이터의 크기(shape) :', questions.shape)
print('A 데이터의 크기(shape) :', answers.shape)
print()
print(questions[0])
print(answers[0])

Q 데이터의 크기(shape) : (11823, 40)
A 데이터의 크기(shape) : (11823, 40)

[8178 7915 4207 3060   41 8179    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]
[8178 3844   74 7894    1 8179    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]


In [8]:
BATCH_SIZE = 64
BUFFER_SIZE = 20000

dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs' : questions,
        'dec_inputs' : answers[:, :-1] # 디코더 입력, 마지막 패딩 토큰 제거
    },
    {
        'outputs' : answers[:, 1:] # 디코더 실제값(교사 강요), 시작 토큰 제거
    },
))

dataset = dataset.cache()
# 전처리 시간을 줄여줌
dataset = dataset.shuffle(BUFFER_SIZE)
# 데이터를 버퍼에 넣고 무작위로 빼내며 셔플, buffer_size가 데이터 크기보다 커야 함
dataset = dataset.batch(BATCH_SIZE)
# 데이터를 batch_size 개수대로 분리
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
# prefetch : 학습중일때, 데이터 로드시간을 줄이기 위해 메모리에 미리 적재시킴
# AUTOTUNE : 네트워크가 알아서 적재시킬 양을 설정해라

print(answers[0]) # 기존 샘플
print(answers[:1][:, :-1]) # 디코더 입력 샘플
print(answers[:1][:, 1:]) # 디코더 정답 샘플

[8178 3844   74 7894    1 8179    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]
[[8178 3844   74 7894    1 8179    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]]
[[3844   74 7894    1 8179    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]]


In [9]:
tf.keras.backend.clear_session()

# 하이퍼파라미터
D_MODEL = 256
NUM_LAYERS = 2
NUM_HEADS = 8
DFF = 512
DROPOUT = 0.1

model = transformer(
    vocab_size = VOCAB_SIZE,
    num_layers = NUM_LAYERS,
    dff = DFF,
    d_model = D_MODEL,
    num_heads = NUM_HEADS,
    dropout = DROPOUT)

learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y, y_hat): # metrics에 적용할 함수
  y = tf.reshape(y, shape=(-1, MAX_LENGTH - 1))
  # -1은 None과 같은 이유, 시작 토큰을 제거했으므로 길이 1 감소
  return tf.keras.metrics.sparse_categorical_accuracy(y, y_hat)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

In [11]:
model.load_weights('./drive/MyDrive/models/DeeDee_weights')

In [ ]:
EPOCHS = 50
model.fit(dataset, epochs=EPOCHS)

In [12]:
def evaluate(sentence):
  sentence = preprocess_sentence(sentence)

  sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)
  # 첫번째 차원 추가
  
  output = tf.expand_dims(START_TOKEN, 0)

  for i in range(MAX_LENGTH):
    predictions = model(inputs=[sentence, output], training=False)

    predictions = predictions[: :, -1:]
    # 문제 발생
    # 해결 : 학습 자료와 다르게 단어 벡터가 세번째 축에 위치했었음
    # 슬라이싱을 세번째 축으로 함으로서 concat 연산이 올바르게 수행되게 함
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    if tf.equal(predicted_id, END_TOKEN[0]):
      break
    
    output = tf.concat([output, predicted_id], axis=-1)

  return tf.squeeze(output, axis=0)

In [13]:
def predict(sentence):
  prediction = evaluate(sentence)

  predicted_sentence = tokenizer.decode(
      [token for token in prediction if token < tokenizer.vocab_size])
  
  print('Input : {}'.format(sentence))
  print('Output : {}'.format(predicted_sentence))

  return predicted_sentence

In [14]:
def preprocess_sentence(sentence):
  sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
  sentence = sentence.strip()
  return sentence

In [15]:
while(True):
  sentence = input()
  if sentence == "exit":
    break
  output = predict(sentence)

안녕
Input : 안녕
Output : 안녕하세요 .
또 만났네
Input : 또 만났네
Output : 좋은 만남이었길 바라요 .
그래 잘가고
Input : 그래 잘가고
Output : 제가 들어드릴게요 .
아니 됐어
Input : 아니 됐어
Output :  !
다음에 보자
Input : 다음에 보자
Output : 잘가요 .
exit


In [41]:
model.save_weights('./drive/MyDrive/models/DeeDee_weights')